In [4]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

from sklearn.feature_extraction.text import CountVectorizer
import pickle
import os
from sklearn import datasets
from sklearn.semi_supervised import label_propagation
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

N_examples = 890393 

def create_dataset():
    y = np.zeros(N_examples)
    i = 0
    corpus = []

    for line in open('train_filtered.csv'):
        label, text = line.split(',')
        corpus.append(text)
        y[i] = int(label)
        i += 1
    
    vectorizer = CountVectorizer(ngram_range=(1, 1),
                                        token_pattern=r'\b\w+\b', min_df=100, max_features=1000)
    
    
    X = vectorizer.fit_transform(corpus)
    
    """
    We change label -1 to 0 and keep -1 to represent unlabeled instances
    """
    y[y==-1] = 0
    
    return X, y

def load_dataset():
    fname = "dataset.pickle"
    if os.path.exists(fname):
        dataset = pickle.load(open(fname, 'rb'))
    else:
        dataset = create_dataset()
        pickle.dump(dataset, open(fname, 'wb'))
    X, y = dataset
    return X, y


In [5]:
"""
    Create reduced dataset
"""
def create_datset_reduced(n_reduced_samples=5000):
    X_full, y_full = load_dataset()
    
    n_total_samples = len(y_full)
    rng = np.random.RandomState(0)
    indices = np.arange(n_total_samples)
    rng.shuffle(indices)
    X = X_full[indices[:n_reduced_samples]]
    y = y_full[indices[:n_reduced_samples]]

    n_labeled_points = 100
    unlabeled_indices = np.arange(n_reduced_samples)[n_labeled_points:]

def load_dataset_reduced():
    fname = "dataset_reduced.pickle"
    if os.path.exists(fname):
        dataset = pickle.load(open(fname, 'rb'))
    else:
        dataset = create_dataset_reduced()
        pickle.dump(dataset, open(fname, 'wb'))
    X, y = dataset
    return X, y


In [10]:
#!/usr/bin/env python3
"""
The script helps guide the users to quickly understand how to use
libact by going through a simple active learning task with clear
descriptions.
"""

import copy
import os

import numpy as np
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split

# libact classes
from libact.base.dataset import Dataset, import_libsvm_sparse
from libact.models import *
from libact.query_strategies import *
from libact.labelers import IdealLabeler


def run(trn_ds, tst_ds, lbr, model, qs, quota, batch_size):
    E_in, E_out = [], []

    batch_i = 0
    for _ in range(quota):
        # Standard usage of libact objects
        ask_id = qs.make_query()
        X, _ = zip(*trn_ds.data)
        lb = lbr.label(X[ask_id])
        trn_ds.update(ask_id, lb)
        batch_i += 1
        if batch_i == batch_size:
            model.train(trn_ds)
            batch_i = 0            
            E_in = np.append(E_in, 1 - model.score(trn_ds))
            E_out = np.append(E_out, 1 - model.score(tst_ds))

    return E_in, E_out


def split_train_test(X, y, test_size, n_labeled):
    X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=test_size)
    trn_ds = Dataset(X_train, np.concatenate(
        [y_train[:n_labeled], [None] * (len(y_train) - n_labeled)]))
    tst_ds = Dataset(X_test, y_test)
    fully_labeled_trn_ds = Dataset(X_train, y_train)

    return trn_ds, tst_ds, y_train, fully_labeled_trn_ds


In [ ]:
# Specifiy the parameters here:

test_size = 0.33    # the percentage of samples in the dataset that will be
                    # randomly selected and assigned to the test set
n_labeled = 100     # number of samples that are initially labeled

# Load dataset
X, y = load_dataset_reduced()
trn_ds, tst_ds, y_train, fully_labeled_trn_ds = \
    split_train_test(X, y, test_size, n_labeled)
trn_ds2 = copy.deepcopy(trn_ds)
lbr = IdealLabeler(fully_labeled_trn_ds)

quota = len(y_train) - n_labeled    # number of samples to query
batch_size = 100

# Comparing UncertaintySampling strategy with RandomSampling.
# model is the base learner, e.g. LogisticRegression, SVM ... etc.
qs = UncertaintySampling(trn_ds, method='lc', model=LogisticRegression(class_weight='balanced'))
model = LogisticRegression(class_weight='balanced')
E_in_1, E_out_1 = run(trn_ds, tst_ds, lbr, model, qs, quota, batch_size)

qs2 = RandomSampling(trn_ds2)
model = LogisticRegression(class_weight='balanced')
E_in_2, E_out_2 = run(trn_ds2, tst_ds, lbr, model, qs2, quota, batch_size)

# Plot the learning curve of UncertaintySampling to RandomSampling
# The x-axis is the number of queries, and the y-axis is the corresponding
# error rate.
query_num = np.arange(1, quota + 1)
plt.plot(query_num, E_in_1, 'b', label='qs Ein')
plt.plot(query_num, E_in_2, 'r', label='random Ein')
plt.plot(query_num, E_out_1, 'g', label='qs Eout')
plt.plot(query_num, E_out_2, 'k', label='random Eout')
plt.xlabel('Number of Queries')
plt.ylabel('Error')
plt.title('Experiment Result')
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05),
           fancybox=True, shadow=True, ncol=5)
plt.show()